In [ ]:
import os

path = os.chdir("..\\..\\Final Project\\datasets\\MNIST")

## Extracting the downloaded files

The downloaded files are in an archive format and needs to be extracted. It can be manually extracted using the GUI or the code segment below can also be used.

In [ ]:
import os,gzip,shutil

files = os.listdir(path)
for file in files:
    if file.endswith('gz'):
        with gzip.open(file, 'rb') as f_in:
            with open(file.split('.')[0], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

In [ ]:
import os,codecs,numpy

files = os.listdir(path)

def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)

data_dict = {}
for file in files:
    if file.endswith('ubyte'):
        with open (file,'rb') as f:
            data = f.read()
            type = get_int(data[:4])
            length = get_int(data[4:8])
            if(type == 2049):
                category = 'labels'
                parsed = numpy.frombuffer(data, dtype=numpy.uint8, offset=8)
                parsed = parsed.reshape(length)
            elif (type == 2051):
                category = 'images'
                num_rows = get_int(data[8:12])
                
                num_cols = get_int(data[12:16])
                parsed = numpy.frombuffer(data,dtype = numpy.uint8, offset = 16)
                parsed = parsed.reshape(length,num_rows,num_cols)
            
            if (length==60000):
                set = 'train'
            elif (length==10000):
                set = 'test'
            data_dict[set+'_'+category] = parsed

In [ ]:
import os
from skimage.io import imsave

folders = ['train','test']

for folder in folders:
    images = data_dict[folder+'_images']
    labels = data_dict[folder+'_labels']
    no_of_samples = images.shape[0]
    for indx in range(no_of_samples):
        image = images[indx]
        label = labels[indx]
        if not os.path.exists(folder+'/'+str(label)+'/'):
            os.makedirs (folder+'/'+str(label)+'/')
        filenumber = len(os.listdir(folder+'/'+str(label)+'/'))
        imsave(folder+'/'+str(label)+'/%05d.png'%(filenumber),image)

In [ ]:
import pickle

with open('MNISTData.pkl', 'wb') as fp :
    pickle.dump(data_dict, fp)

with open('MNISTData.pkl', 'rb') as fp :
    new_dict = pickle.load(fp)